In [13]:
VER=3

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import gc
pd.set_option('display.max_columns', 500)

In [14]:
def compute_rmse(predictions, target_values):
    """Calculate RMSE between predictions and true values"""
    return np.sqrt(np.mean((predictions - target_values)**2.0))

def compute_multiple_rmse(target_values, prediction_matrix):
    """
    Computes multiple RMSE scores simultaneously for multiple model predictions.
    
    Parameters:
    ----------
    target_values : numpy.ndarray - True values (N samples)
    prediction_matrix : numpy.ndarray - Predictions from multiple models (N samples, K models)
    
    Returns:
    -------
    numpy.ndarray - RMSE scores for each model (K scores)
    """
    if len(target_values.shape) == 1:
        target_values = target_values[:, np.newaxis]
    return np.sqrt(np.mean((target_values - prediction_matrix)**2.0, axis=0))

In [27]:
# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")
# train_og = pd.read_csv('calories.csv')
# train_og = train_og.rename(columns={'Gender': 'Sex', 'User_ID': 'id'})
# train = pd.concat([train, train_og])

meta_train = pd.read_csv("meta_train_v1.csv")
meta_test = pd.read_csv("meta_test_v1.csv")

true = np.log1p(train.Calories.values)

In [28]:
meta_train2 = pd.read_csv("meta_train.csv")
meta_test2 = pd.read_csv("meta_test.csv")

# add postfix to column names
meta_train2.columns = [f"{col}_2" for col in meta_train2.columns]
meta_test2.columns = [f"{col}_2" for col in meta_test2.columns]

# add meta_train2 to meta_train
meta_train = pd.concat([meta_train, meta_train2], axis=1)
meta_test = pd.concat([meta_test, meta_test2], axis=1)

In [29]:
# meta_test['id'] = meta_test.index

# Load additional meta features
additional_model1 = pd.read_csv("may2025-playground-oofs-testpreds/submission_cat.csv")
additional_model1['Calories'] = np.log1p(additional_model1['Calories'])
additional_model1.rename(columns={'Calories': 'addcat_preds'}, inplace=True)

additional_model2 = pd.read_csv("may2025-playground-oofs-testpreds/submission_nn.csv")
additional_model2['Calories'] = np.log1p(additional_model2['Calories'])
additional_model2.rename(columns={'Calories': 'addnn_preds'}, inplace=True)

additional_model3 = pd.read_csv("may2025-playground-oofs-testpreds/submission_xgb.csv")
additional_model3['Calories'] = np.log1p(additional_model3['Calories'])
additional_model3.rename(columns={'Calories': 'addxgb_preds'}, inplace=True)

# Merge additional models into meta_train and meta_test
meta_test['addcat_preds'] = additional_model1['addcat_preds']
meta_test['addnn_preds'] = additional_model2['addnn_preds']
meta_test['addxgb_preds'] = additional_model3['addxgb_preds']

meta_test

,lgb_pred,xgb_pred,extra_pred,cat_pred,ridge_pred,linreg_pred,mlpr_pred,mlpr2_pred,lgb_pred_2,xgb_pred_2,extra_pred_2,cat_pred_2,ridge_pred_2,linreg_pred_2,mlpr_pred_2,addcat_preds,addnn_preds,addxgb_preds
0,3.344351,3.342569,3.348434,3.338018,3.349798,-1.626867e+09,3.264989,3.321397,3.347372,3.344998,3.349441,3.347327,3.348717,3.347646,3.344878,3.359586,3.349408,3.348672
1,4.684991,4.695809,4.676702,4.688298,4.703145,-1.626867e+09,4.606328,4.660454,4.689246,4.696438,4.681854,4.692392,4.699879,4.703234,4.694859,4.688181,4.684138,4.688597
2,4.487348,4.479144,4.492839,4.481136,4.496058,-5.050733e+08,4.522970,4.459732,4.493549,4.484334,4.494137,4.475586,4.494918,4.495127,4.470546,4.491739,4.476347,4.483576
3,4.841115,4.843299,4.855392,4.847232,4.848108,-5.050733e+08,4.899315,4.844353,4.838822,4.844844,4.857342,4.847325,4.851188,4.848032,4.844933,4.834117,4.832845,4.843349
4,4.336153,4.341221,4.339494,4.343300,4.341677,-5.050733e+08,4.406139,4.342793,4.339240,4.341553,4.340597,4.345902,4.335855,4.337154,4.346213,4.345689,4.341462,4.341073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,3.303092,3.308285,3.295263,3.299477,3.301017,-5.050733e+08,3.356085,3.300908,3.302131,3.306865,3.297504,3.298242,3.296079,3.296763,3.302865,3.301077,3.297971,3.297824
249996,2.357912,2.355627,2.327494,2.369028,2.352110,-1.626867e+09,2.279192,2.324196,2.355985,2.367482,2.331657,2.363635,2.358946,2.366800,2.336775,2.351698,2.342908,2.343007
249997,4.309456,4.305654,4.303814,4.304719,4.309337,-5.050733e+08,4.365372,4.306429,4.305157,4.303197,4.303239,4.304199,4.309983,4.312088,4.310526,4.302713,4.305157,4.304109
249998,5.128446,5.129230,5.131132,5.128062,5.146676,-5.050733e+08,5.193858,5.131546,5.132790,5.130076,5.130999,5.131405,5.143945,5.143106,5.130893,5.128648,5.129272,5.137757


In [31]:
# load npy files
meta_train['addcat_preds'] = np.load("may2025-playground-oofs-testpreds/oof_cat.npy")
meta_train['addnn_preds'] = np.load("may2025-playground-oofs-testpreds/oof_nn.npy")
meta_train['addxgb_preds'] = np.load("may2025-playground-oofs-testpreds/oof_xgb.npy")
meta_train

,lgb_pred,xgb_pred,extra_pred,cat_pred,ridge_pred,linreg_pred,mlpr_pred,mlpr2_pred,lgb_pred_2,xgb_pred_2,extra_pred_2,cat_pred_2,ridge_pred_2,linreg_pred_2,mlpr_pred_2,addcat_preds,addnn_preds,addxgb_preds
0,4.998588,5.000219,4.992605,5.003390,4.968647,4.964198,5.002721,4.998046,5.007776,5.002449,4.995192,4.990682,4.990530,4.996628,5.009289,4.995950,5.005943,4.991400
1,3.612345,3.599840,3.607457,3.608462,3.587455,3.583986,3.624764,3.610288,3.589667,3.600411,3.606644,3.622508,3.588695,3.583401,3.622752,3.604675,3.609127,3.604128
2,3.404320,3.402904,3.396705,3.405091,3.405076,3.400074,3.400764,3.399783,3.392132,3.401425,3.402125,3.415250,3.400481,3.398516,3.393176,3.411715,3.405542,3.407911
3,4.934748,4.935765,4.936412,4.928371,4.928038,4.928436,4.933142,4.943055,4.936567,4.934491,4.935363,4.928582,4.924314,4.921657,4.942299,4.925594,4.938608,4.942559
4,4.994274,4.990899,4.987925,5.000328,4.991310,4.995078,4.991543,4.985012,4.986480,4.992103,4.989324,4.994293,4.989660,4.990102,4.993887,4.994617,4.989333,4.989571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,5.399784,5.399776,5.386666,5.412525,5.411143,5.415754,5.403661,5.420692,5.399846,5.399239,5.387854,5.409313,5.401943,5.404689,5.396260,5.408215,5.411156,5.388842
749996,4.555580,4.557221,4.564021,4.551572,4.550077,4.550217,4.553001,4.563708,4.570198,4.557627,4.564100,4.555979,4.550867,4.548317,4.552946,4.564720,4.557039,4.548698
749997,5.452100,5.449432,5.454759,5.463372,5.472982,5.470604,5.456325,5.463422,5.458003,5.458263,5.443864,5.464071,5.470189,5.471018,5.472954,5.465971,5.450336,5.446399
749998,4.681223,4.682672,4.679208,4.686352,4.687036,4.691645,4.695790,4.678912,4.688296,4.683241,4.679833,4.680248,4.691381,4.686368,4.674783,4.679418,4.670905,4.674219


In [32]:
# Model prediction columns from the DataFrame
model_columns = meta_train.columns.tolist()
files = model_columns  # Using column names as "file" names for the models

In [33]:
# Extract predictions from DataFrame to numpy arrays
x_train = meta_train[model_columns].values
x_test = meta_test[model_columns].values

print("Our combined OOF have shape:", x_train.shape)
print("Our combined PRED have shape:", x_test.shape)

Our combined OOF have shape: (750000, 18)
Our combined PRED have shape: (250000, 18)


In [34]:
# Setup for hill climbing
print("\n## Setting Up Hill Climbing")
ALLOW_NEGATIVE_WEIGHTS = True
MAX_ENSEMBLE_SIZE = 1000
IMPROVEMENT_THRESHOLD = 1e-6

# Hill Climbing Ensemble Optimization
print("\n" + "#"*50)
print("# Hill Climbing Ensemble Optimization")
print("#"*50)

# Convert data to numpy for processing
model_predictions = np.array(x_train)
test_predictions = np.array(x_test)
log_target = np.array(true)


## Setting Up Hill Climbing

##################################################
# Hill Climbing Ensemble Optimization
##################################################


In [35]:
# Find best individual model as starting point
print("\n## Finding Best Base Model")
best_single_rmse = 40
best_model_index = -1

for model_idx, model_name in enumerate(files):
    single_model_rmse = compute_rmse(model_predictions[:, model_idx], log_target)
    if single_model_rmse < best_single_rmse:
        best_single_rmse = single_model_rmse
        best_model_index = model_idx
    print(f"RMSE {single_model_rmse:0.5f} {model_name}")

print(f"\nBest single model is {files[best_model_index]} with RMSE = {best_single_rmse:0.5f}")



## Finding Best Base Model
RMSE 0.06016 lgb_pred
RMSE 0.05966 xgb_pred
RMSE 0.06032 extra_pred
RMSE 0.05997 cat_pred
RMSE 0.06886 ridge_pred
RMSE 0.06869 linreg_pred
RMSE 0.06206 mlpr_pred
RMSE 0.06203 mlpr2_pred
RMSE 0.06028 lgb_pred_2
RMSE 0.05964 xgb_pred_2
RMSE 0.06032 extra_pred_2
RMSE 0.05994 cat_pred_2
RMSE 0.06873 ridge_pred_2
RMSE 0.06861 linreg_pred_2
RMSE 0.06153 mlpr_pred_2
RMSE 0.05982 addcat_preds
RMSE 0.06096 addnn_preds
RMSE 0.05988 addxgb_preds

Best single model is xgb_pred_2 with RMSE = 0.05964


In [36]:
# Track the best models and weights
selected_indices = [best_model_index]
selected_models = [best_model_index]
selected_weights = []
improvement_history = [best_single_rmse]
previous_best_rmse = best_single_rmse

# Initialize with best single model predictions
current_ensemble_preds = model_predictions[:, best_model_index].copy()

# Define weight range to explore
min_weight = -0.50 if ALLOW_NEGATIVE_WEIGHTS else 0.01
weight_range = np.arange(min_weight, 0.51, 0.005)
n_weight_steps = len(weight_range)

print(f"Starting with: {files[best_model_index]} (RMSE: {best_single_rmse:0.5f})")
print(f"Exploring weights from {min_weight} to 0.5 with {n_weight_steps} steps")
print(f"Maximum models in ensemble: {MAX_ENSEMBLE_SIZE}")
print(f"Improvement threshold: {IMPROVEMENT_THRESHOLD}")

# Begin hill climbing process
print("\n## Hill Climbing Iteration Process")
for iteration in range(1_000_000):
    # Reset best values for this iteration
    iteration_best_rmse = 40
    iteration_best_model = -1
    iteration_best_weight = 0
    iteration_best_predictions = None

    # Try adding each model to the current ensemble
    for model_idx, model_name in enumerate(files):
        # Get candidate model predictions
        candidate_model_preds = model_predictions[:, model_idx]

        # Create weighted combinations across all weight values
        # Current ensemble gets weight (1-w), new model gets weight (w)
        weighted_ensemble = np.repeat(current_ensemble_preds[:, np.newaxis], n_weight_steps, axis=1) * (1-weight_range)
        weighted_candidate = np.repeat(candidate_model_preds[:, np.newaxis], n_weight_steps, axis=1) * weight_range
        combined_predictions = weighted_ensemble + weighted_candidate

        # Calculate RMSE for all weight combinations
        rmse_scores = compute_multiple_rmse(log_target, combined_predictions)
        candidate_rmse = np.min(rmse_scores).item()

        # Check if this is the best combination found so far
        if candidate_rmse < iteration_best_rmse:
            iteration_best_rmse = candidate_rmse
            iteration_best_model = model_idx
            best_weight_idx = np.argmin(rmse_scores).item()
            iteration_best_weight = weight_range[best_weight_idx].item()
            iteration_best_predictions = combined_predictions[:, best_weight_idx]

    # Clean up memory
    del candidate_model_preds, weighted_ensemble, weighted_candidate, combined_predictions, rmse_scores
    gc.collect()

    # Check stopping criteria
    selected_indices.append(iteration_best_model)
    selected_indices = list(np.unique(selected_indices))

    if len(selected_indices) > MAX_ENSEMBLE_SIZE:
        print(f"=> Stopped: Reached maximum of {MAX_ENSEMBLE_SIZE} models")
        selected_indices = selected_indices[:-1]
        break

    improvement = previous_best_rmse - iteration_best_rmse
    if improvement < IMPROVEMENT_THRESHOLD:
        print(f"=> Stopped: Improvement ({improvement:0.6f}) below threshold ({IMPROVEMENT_THRESHOLD})")
        break

    # Record new result
    print(f"Iteration {iteration+1}: RMSE {iteration_best_rmse:0.5f} adding "
          f"\"{files[iteration_best_model]}\" with weight {iteration_best_weight:0.3f} "
          f"(Improvement: {improvement:0.6f})")

    selected_models.append(iteration_best_model)
    selected_weights.append(iteration_best_weight)
    improvement_history.append(iteration_best_rmse)
    current_ensemble_preds = iteration_best_predictions
    previous_best_rmse = iteration_best_rmse

# Calculate final model weights
print("\n## Calculating Final Ensemble Weights")
# Initialize first model weight as 1
final_weights = np.array([1])

# Calculate remaining weights
for weight in selected_weights:
    # Update prior model weights
    final_weights = final_weights * (1-weight)
    # Add new model weight
    final_weights = np.concatenate([final_weights, np.array([weight])])

# Create summary DataFrame
print("\n## Creating Weight Summary")
summary_rows = []
total_weight = 0

for model_idx, weight, rmse in zip(selected_models, final_weights, improvement_history):
    model_name = files[model_idx]
    summary_rows.append({
        'model': model_name,
        'weight': weight,
        'rmse': rmse
    })
    total_weight += float(f'{weight:.3f}')

# Display model weights summary
weight_summary = pd.DataFrame(summary_rows)
weight_summary = weight_summary.groupby('model').agg('sum').reset_index().sort_values('weight', ascending=False)
weight_summary = weight_summary.reset_index(drop=True)
print(weight_summary)

# Verify weights sum approximately to 1
print(f'Ensemble weights sum to {weight_summary.weight.sum():0.5f}')

Starting with: xgb_pred_2 (RMSE: 0.05964)
Exploring weights from -0.5 to 0.5 with 202 steps
Maximum models in ensemble: 1000
Improvement threshold: 1e-06

## Hill Climbing Iteration Process
Iteration 1: RMSE 0.05935 adding "addcat_preds" with weight 0.440 (Improvement: 0.000289)
Iteration 2: RMSE 0.05926 adding "addxgb_preds" with weight 0.275 (Improvement: 0.000089)
Iteration 3: RMSE 0.05919 adding "cat_pred_2" with weight 0.230 (Improvement: 0.000067)
Iteration 4: RMSE 0.05917 adding "extra_pred" with weight 0.130 (Improvement: 0.000025)
Iteration 5: RMSE 0.05915 adding "addnn_preds" with weight 0.085 (Improvement: 0.000017)
Iteration 6: RMSE 0.05914 adding "lgb_pred_2" with weight 0.090 (Improvement: 0.000012)
Iteration 7: RMSE 0.05913 adding "ridge_pred" with weight -0.025 (Improvement: 0.000006)
Iteration 8: RMSE 0.05913 adding "cat_pred" with weight 0.050 (Improvement: 0.000002)
Iteration 9: RMSE 0.05913 adding "addxgb_preds" with weight 0.060 (Improvement: 0.000003)
Iteration 10

In [37]:
# Generate final predictions
print("\n## Generating Final Predictions")
# Create model name to index mapping
model_to_index_map = {model_name: idx for idx, model_name in enumerate(files)}

# Combine OOF predictions with optimal weights
ensemble_oof_preds = model_predictions[:, model_to_index_map[weight_summary.model.iloc[0]]] * weight_summary.weight.iloc[0]
for k in range(1, len(weight_summary)):
    model_idx = model_to_index_map[weight_summary.model.iloc[k]]
    weight = weight_summary.weight.iloc[k]
    ensemble_oof_preds += model_predictions[:, model_idx] * weight

# Calculate final ensemble RMSE
final_ensemble_rmse = compute_rmse(ensemble_oof_preds, log_target)
print(f'Final Ensemble RMSE = {final_ensemble_rmse:0.6f}')


## Generating Final Predictions
Final Ensemble RMSE = 0.059123


In [38]:
ensemble_oof_preds

array([4.99826375, 3.60707402, 3.40522549, ..., 5.45727224, 4.6794882 ,
       4.62064552])

In [39]:
# Save OOF predictions
np.save(f'oof_hill_climb_v{VER}', ensemble_oof_preds)

# Create test predictions
ensemble_test_preds = test_predictions[:, model_to_index_map[weight_summary.model.iloc[0]]] * weight_summary.weight.iloc[0]
for k in range(1, len(weight_summary)):
    model_idx = model_to_index_map[weight_summary.model.iloc[k]]
    weight = weight_summary.weight.iloc[k]
    ensemble_test_preds += test_predictions[:, model_idx] * weight

# Prepare submission
print("\n## Creating Submission File")
submission = pd.read_csv("sample_submission.csv")

# Clip predictions to train min and max
min_value = train.Calories.min()
max_value = train.Calories.max()
submission.Calories = np.clip(np.expm1(ensemble_test_preds), min_value, max_value)

print(f"Test shape: {submission.shape}")
print(f"Test target mean: {submission.Calories.mean():0.4f}")
submission.to_csv(f"submission_hill_climb_v{VER}.csv", index=False)


## Creating Submission File
Test shape: (250000, 2)
Test target mean: 88.1480


In [40]:
submission

,id,Calories
0,750000,27.461741
1,750001,107.739804
2,750002,87.621179
3,750003,125.836200
4,750004,75.884811
...,...,...
249995,999995,26.128920
249996,999996,9.516028
249997,999997,73.010125
249998,999998,168.228125
